In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
from collections import defaultdict

from utils.aerospike_analyzer import *
from utils.plot import *

In [ ]:
BASE_PATH = Path('..', 'aerospike')
BATCH = 10
NUM_CORES_LIST = [1, 2, 4, 8, 16]

DRAM_LATENCY_LIST = [0]
CXL_LATENCY_LIST = [500, 1000, 1500, 2000, 2500, 3000, 4000, 5000, 10000, 20000]
MEM_LATENCY_LIST = DRAM_LATENCY_LIST + CXL_LATENCY_LIST

In [ ]:
max_throughput_dict = {}
for num_cores in NUM_CORES_LIST:
    dram_log_dir = BASE_PATH.joinpath('run-act-ro-dram-%dcore-log' % num_cores)
    cxl_log_dir = BASE_PATH.joinpath('run-act-ro-cxl-%dcore-log' % num_cores)
    data = parse_logs(dram_log_dir.glob('act-*.log'), DRAM_LATENCY_LIST)
    d = parse_logs(cxl_log_dir.glob('act-*.log'), CXL_LATENCY_LIST)
    data.update(d)
    max_throughput_list = []
    for mem_latency in MEM_LATENCY_LIST:
        max_t, _, _, _, _, _ = analyze_given_mem_latency_and_batch(data, mem_latency, BATCH)
        max_throughput_list.append(max_t)
    max_throughput_dict[num_cores] = max_throughput_list

In [ ]:
SWITCH_TIME = 50
NUM_PREFETCHES = 12
NUM_CHASES = 8
MEMORY_TIME = 139.5 - 62
IO_TIME_PRE = 2009.7 - 62
IO_TIME_POST = 1292.1 - 62
TITLE = 'Aerospike (500M items, single core, read-only)'
NAME = 'aerospike500m_1core_rdonly'
NUM_CORES = 1

latencies = []
throughputs = []
for lat, t in zip(MEM_LATENCY_LIST, max_throughput_dict[NUM_CORES]):
    if(lat > 1000 and lat % 1000 == 500):
        continue
    elif(lat > 15000):
        continue
    else:
        latencies.append(lat)
        throughputs.append(t)

_ = plot_with_models(latencies, throughputs,
                     NUM_CHASES, MEMORY_TIME, IO_TIME_PRE, IO_TIME_POST,
                     SWITCH_TIME, NUM_PREFETCHES, 'Aerospike/CacheLib', TITLE, NAME)

In [ ]:
throughput_dict = defaultdict(list)
for i, lat in enumerate(MEM_LATENCY_LIST):
    if(lat > 1000 and lat % 1000 == 500):
        continue
    elif(lat > 15000):
        continue
    else:
        for num_cores in NUM_CORES_LIST:
            throughput_dict[lat].append(max_throughput_dict[num_cores][i])

with open('aerospike.json', 'w') as f:
    json.dump(throughput_dict, f, indent=4)

plot_core_scaling(NUM_CORES_LIST, throughput_dict, 'Aerospike (500M items, read-only)')

In [ ]:
NUM_CORES = 16
plot_throughputs(MEM_LATENCY_LIST, max_throughput_dict[NUM_CORES], 2.1, 'C0',
                 'Aerospike (500M items, read-only)', 'aerospike500m_16core_rdonly')

In [ ]:
DRAM_LOG_DIR = BASE_PATH.joinpath('run-act-rw-dram-8core-log')
CXL_LOG_DIR  = BASE_PATH.joinpath('run-act-rw-cxl-8core-log')

data = parse_logs(DRAM_LOG_DIR.glob('act-*.log'), DRAM_LATENCY_LIST)
d    = parse_logs(CXL_LOG_DIR.glob('act-*.log'), CXL_LATENCY_LIST)
data.update(d)

In [ ]:
def plot_all_throughputs(mem_latency, d):
    client_threads_list = sorted(list(d.keys()))
    server_threads_list = sorted(list(d[client_threads_list[0]].keys()))
    x = np.arange(len(server_threads_list))
    plt.figure(figsize=(15,5))
    n = len(client_threads_list)
    width = 0.8 / n
    for i, c_th in enumerate(client_threads_list):
        y = []
        for s_th in server_threads_list:
            if(s_th in d[c_th]):
                y.append(d[c_th][s_th])
            else:
                y.append(0)
        plt.bar(x + (i - n/2 + 0.5) * width, y, width=width*0.8, label='%d clients' % c_th)
    plt.xticks(x, server_threads_list)
    plt.xlabel('Number of server threads')
    plt.ylabel('Throughput [ops/sec]')
    if(mem_latency == 0):
        plt.title('DRAM 0.1 usec')
    else:
        plt.title('CXL %.1f usec' % (mem_latency / 1000))
    plt.legend(bbox_to_anchor=(1, 0), loc='lower left')
    plt.show()

In [ ]:
BATCH = 10
SERVER_THREADS = 512
CLIENT_THREADS = 512

throughput_list = []
rd_latency_list = []
wr_latency_list = []
for mem_latency in MEM_LATENCY_LIST:
    _, _, _, t, r, w = analyze_given_mem_latency_and_batch(data, mem_latency, BATCH)
    plot_all_throughputs(mem_latency, t)
    throughput_list.append(t[CLIENT_THREADS][SERVER_THREADS])
    rd_latency_list.append(r[CLIENT_THREADS][SERVER_THREADS])
    wr_latency_list.append(w[CLIENT_THREADS][SERVER_THREADS])

plot_throughputs(MEM_LATENCY_LIST, throughput_list, None, '#9400d3',
                 'Aerospike Operation Throughput (500M items, read-write mix)', 'aerospike500m_8core')

plot_latencies(MEM_LATENCY_LIST, rd_latency_list, [50, 90, 99], BATCH,
               'Aerospike Operation Latency (500M items, read-write mix)', 'aerospike500m_8core_latency')